In [0]:
import requests
from IPython.core.display import HTML
HTML(f"""
<style>
@import "https://cdn.jsdelivr.net/npm/bulma@0.9.4/css/bulma.min.css";
</style>
""")

# Descriptive Statistics and Handling Missing Data in Pose Data Sequences
This exercise delves into the exploration of descriptive statistics and methodologies for managing missing data, particularly within a slightly intricate dataset comprising fixed-length skeleton data sequences. Moreover, it encompasses the evaluation of the efficacy of these methodologies.
The primary objective is to implement and scrutinize various techniques for managing absent data points, ensuring a comprehensive and unblemished dataset for subsequent analyses.
### Pose Data Overview
In the subsequent cell, the ground truth dataset is loaded alongside an identical yet noise-infused counterpart. This involves a 30-frame time series sequence, capturing human poses with each frame encompassing 25_2 features, namely the x and y coordinates for 25 skeletal joints. We'll generate 200 such sequences, forming a dataset with a shape of ( (200, 30, 25_2) ).
#### Handling Missing Values
From the ground truth dataset (`GT_poses`
), data has been intentionally omitted, thereby creating the `Noisy_poses`
 dataset wherein 30% of the original values have been randomly substituted with NaN values (`np.nan`
). The ensuing step involves implementing and evaluating varied strategies for dealing with these missing data points, ensuring the robustness of the eventual analyses derived from this dataset.

<article class="message">
    <div class="message-body">
        <strong>List of tasks</strong>
        <ul style="list-style: none;">
            <li>
            <a href="#filling_zeros">Task 1: Filling missing data - zeros</a>
            </li>
            <li>
            <a href="#mean_impute">Task 2: Mean imputer</a>
            </li>
            <li>
            <a href="#temp_interpolation">Task 3: Temporal interpolation</a>
            </li>
            <li>
            <a href="#evaluation">Task 4: Adjust data and evaluate</a>
            </li>
            <li>
            <a href="#method_comp">Task 5: Method comparison</a>
            </li>
            <li>
            <a href="#Mean">Task 6: Mean pose and pose sequence</a>
            </li>
            <li>
            <a href="#Correlation">Task 7: Correlation between features</a>
            </li>
        </ul>
    </div>
</article>



In [0]:
## Load the data 
# give a title - to this box
## import libraries
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use('seaborn')
np.random.seed(42)
GT_poses  = np.load("data/Input_Poses.npy")
Noisy_poses = np.load('data/Noisy_poses.npy')

N,T,D,C = Noisy_poses.shape
GT_poses = GT_poses.reshape(N,T,D*C)
Noisy_poses = Noisy_poses.reshape(N,T,D*C)

Helper plotting functions in the cell below. It is not required to understand or modify any of the plotting functions to complete the exercise.


In [0]:
def limb_number_plot(s_pose_x,s_pose_y,n1,n2,c="red",label=None):
  if label is not None:
    if (s_pose_x[n1]>0) and (s_pose_x[n2]>0) and (s_pose_y[n1]>0) and (s_pose_y[n2]>0): 
      plt.plot([s_pose_x[n1],s_pose_x[n2]], [s_pose_y[n1], s_pose_y[n2]],color = c, linestyle="-",label=label)
  else:
    if (s_pose_x[n1]>0) and (s_pose_y[n1]>0):
       plt.plot(s_pose_x[n1], s_pose_y[n1],'*',color = c,label=label)
    if (s_pose_x[n2]>0) and (s_pose_y[n2]>0):
       plt.plot(s_pose_x[n2], s_pose_y[n2],'*',color = c,label=label)
    if (s_pose_x[n1]>0) and (s_pose_x[n2]>0) and (s_pose_y[n1]>0) and (s_pose_y[n2]>0):
      plt.plot([s_pose_x[n1],s_pose_x[n2]], [s_pose_y[n1], s_pose_y[n2]],color = c, linestyle="-")

def plot_single_pose(s_pose,c = "darkgreen",label=None,ds='body_25',c_head = 'red',head = True):
    
    s_pose_x=s_pose[::2]
    s_pose_y=s_pose[1::2]
    #torso/body
    limb_number_plot(s_pose_x,s_pose_y,2,5,c)
    if label is not None:

        limb_number_plot(s_pose_x,s_pose_y,9,12,c,label)
    else:
        limb_number_plot(s_pose_x,s_pose_y,9,12,c)
    limb_number_plot(s_pose_x,s_pose_y,2,9,c)
    limb_number_plot(s_pose_x,s_pose_y,5,12,c)

    #left arm (person facing away)
    limb_number_plot(s_pose_x,s_pose_y,2,3,c)
    limb_number_plot(s_pose_x,s_pose_y,3,4,c)

    #right arm
    limb_number_plot(s_pose_x,s_pose_y,5,6,c)
    limb_number_plot(s_pose_x,s_pose_y,6,7,c)

    #left leg / foot
    limb_number_plot(s_pose_x,s_pose_y,9,10,c)
    limb_number_plot(s_pose_x,s_pose_y,10,11,c)
    limb_number_plot(s_pose_x,s_pose_y,11,22,c)
    #right leg / foot
    limb_number_plot(s_pose_x,s_pose_y,12,13,c)
    limb_number_plot(s_pose_x,s_pose_y,13,14,c)
    limb_number_plot(s_pose_x,s_pose_y,14,19,c)

    # head
    if head:
        limb_number_plot(s_pose_x,s_pose_y,0,15,c)
        limb_number_plot(s_pose_x,s_pose_y,0,16,c)

        limb_number_plot(s_pose_x,s_pose_y,15,17,c)
        limb_number_plot(s_pose_x,s_pose_y,16,18,c)
    return True 

def plot_single_sequence(poses, pose_name='Poses',color='blue'):
    """
    Plots a single sequence of skeleton joints.

    Parameters:
        poses (array-like): Skeleton sequence data, shape (T,D).
        poses_name (string, optional): subtitle of each skeleton body in the sequence. 
        color (string, optional): color of skeleton bodies. 
    """
    plt.style.use('seaborn')
    plt.figure(figsize=(20,5))
    plt.title('Ground truth')

    for i in range(len(poses)):
        plt.subplot(3, 10, i + 1)
        plot_single_pose(poses[i], c=color, head=False)
        plt.ylim(1, 0)
        plt.xlim(-1, 1)
        plt.title(pose_name + str(i))
        plt.axis('off')

    plt.show()

In [0]:
## Plot a random single sequence of both the noisy and ground truth skeleton data.
np.random.seed(12)
sample = np.random.randint(0,200)
plot_single_sequence(GT_poses[sample],pose_name='GT',color='blue')
plot_single_sequence(Noisy_poses[sample],pose_name='Noisy poses',color='red')

### Missing data strategies
In the following steps we will learn 3 different methods for handling missing data. 
1. Inserting zeros
2. Inserting means
3. Linear interpolation over time

<article class="message task"><a class="anchor" id="filling_zeros"></a>
    <div class="message-header">
        <span>Task 1: Filling missing data - zeros</span>
        <span class="has-text-right">
          <i class="bi bi-code"></i><i class="bi bi-stoplights easy"></i>
        </span>
    </div>
<div class="message-body">


Implement the first of 3 different methods for filling in missing data in the `Noisy_poses`
 in the cell below. The approach `filling_missing`
 should fill missing entries in the data sequences (`np.nan`
-values)with a zeros.
**Note:** (Hint: the function `np.isnan`
 can be used to find indices containing NaN values.)



</div></article>



In [0]:
## Method 1: Flling missing entries with 0s
def filling_missing(data):
    """
    Fills missing entries in the given data with a specified value.

    Parameters:
    - data (numpy.array): 3D numpy array with shape (num_samples, seq_len, num_features) containing data entries.
    - filling (int or float): The value used to fill missing entries.

    Returns:
    - numpy.array: Data with missing entries filled using the specified value.
    """
    #write code here ...

<article class="message task"><a class="anchor" id="mean_impute"></a>
    <div class="message-header">
        <span>Task 2: Mean imputer</span>
        <span class="has-text-right">
          <i class="bi bi-code"></i><i class="bi bi-stoplights medium"></i>
        </span>
    </div>
<div class="message-body">


Implement the `mean_imputer`
 function below. The function should contain:
- Extract `data`
 shape and create a copy, `imputed_data`
, for imputation.

- Loop through each feature in `imputed_data`
.

- Compute the mean, identify `np.nan`
 indices, and replace them with the mean.

- Ensure `imputed_data`
 is returned, with all `np.nan`
 entries imputed.




</div></article>



In [0]:
## Method 2: Mean impute
def mean_imputer(data):
    """
    Imputes missing entries in the given data using the mean value of the features.

    Parameters:
    - data (numpy.array): 3D numpy array with shape (num_samples, sequence_len, num_features) containing data entries.

    Returns:
    - numpy.array: Data with missing entries imputed using the mean value.
    """
    #write code here ...

<article class="message task"><a class="anchor" id="temp_interpolation"></a>
    <div class="message-header">
        <span>Task 3: Temporal interpolation <em>(optional)</em></span>
        <span class="has-text-right">
          <i class="bi bi-code"></i><i class="bi bi-stoplights hard"></i>
        </span>
    </div>
<div class="message-body">


Implementing the `time_series_interpolation`
 Function: A Stepwise Exercise,   - Define the function `time_series_interpolation(data)`
 with `data`
 as a 3D numpy array input.
- Extract the shape of `data`
 and create a copy named `interpolated_data`
 to perform operations without altering the original data.

- Implement nested for-loops to iterate through each sample (i) and each feature (j) in `interpolated_data`
.

- Inside the nested loops:
    - Retrieve the time series sequence for the current sample and feature.
    - Handle cases for the first element being `np.nan`
 by finding the next available value in the sequence and imputing the `np.nan`
 using a suitable replacement logic.
    - Iterate through the time series (k), locating `np.nan`
 entries, and implementing logic to interpolate values, ensuring handling of consecutive `np.nan`
 entries.
    - Account for the scenario where the last element is `np.nan`
, replacing it with the second-to-last element.


- Have the function return `interpolated_data`
 with all `np.nan`
 entries interpolated.




</div></article>



In [0]:
## Mehtod 3:Temporal approach
def time_series_interpolation(data):
    """
    Performs time series interpolation to fill missing entries in the given data.

    Parameters:
    - data (numpy.array): 3D numpy array with shape (num_samples, sequence_len, num_features) containing data entries.

    Returns:
    - numpy.array: Data with missing entries filled using time series interpolation.
    """
    #write code here ...

### Apply and evaluate approaches
#### Error function (MSE)
The function  `MSE`
 that find the mean squared error between the adjusted and ground truth data.
- the MSE can be averaged across different dimensions of the skeleton pose data array such that average loss across each step of the time seqeunce ('Temporal') or the loss across each joints/features ('Joint') is calculated.


In [0]:
def MSE(Noisy,GT,avg='Temporal'):
    loss = (Noisy - GT) ** 2
    if avg=='Temporal':
        loss = loss.mean(axis=(0,2))
    elif avg == "Joint":
        loss = loss.mean(axis=(0,1))
    else:
        loss  = loss.mean()
    return loss

<article class="message task"><a class="anchor" id="evaluation"></a>
    <div class="message-header">
        <span>Task 4: Adjust data and evaluate</span>
        <span class="has-text-right">
          <i class="bi bi-code"></i><i class="bi bi-stoplights easy"></i>
        </span>
    </div>
<div class="message-body">


Evaluate the performance of each imputation approach by comparing the adjusted data to the ground truth seqeunces using the `MSE`
 function. The task is done in the following steps.
1. Adjust the noisy data with an imputation method.
2. Calculate the _mean squared error_ between the adjusted and ground truth poses, by using the function `MSE`
. 
3. Repeat for the remaining methods.



</div></article>



In [0]:
### Apply the 3 missing data measures to the noisy data and calculate their respective MSE
##example
#adjusted_data1 =  filling_missing(Noisy_poses)
#error1 = MSE(adjusted_data1,GT_poses,avg='None')
#print(f'The MSE of the data adjusted with filling in zeros: {error1}')

### Plotting
#### Plot error distances
In the cell below, the MSE is plotted between the ground truth and adjusted poses, averaging both over the joints and the temporal sequence.


In [0]:
## Your should only run this cell after completing the exercise above
Dist1 = MSE(adjusted_data3,GT_poses)
plt.figure(figsize=(20,10))
plt.title('Distance Pose seqeunce')
plt.subplot(2,1,1)
plt.bar(np.arange(len(Dist1)), Dist1, align='center',label='Temporal Distance')
plt.legend()
plt.subplot(2,1,2)
Dist2 = MSE(adjusted_data3,GT_poses,avg='Joint')
plt.bar(np.arange(len(Dist2)), Dist2, align='center',color='green',label='Joint Distance')
plt.legend();

#### Plotting GT, noisy and adjusted poses


In [0]:
## Your should only run this cell after completing the exercise above
## Plot a random single sequence of the noisy, ground truth and adjusted skeleton data.
np.random.seed(42)
sample = np.random.randint(0,200)
plot_single_sequence(GT_poses[sample],pose_name='GT',color='blue')
plot_single_sequence(Noisy_poses[sample],pose_name='Noisy poses',color='red')
plot_single_sequence(adjusted_data3[sample],pose_name='Adjusted poses',color='green')

<article class="message task"><a class="anchor" id="method_comp"></a>
    <div class="message-header">
        <span>Task 5: Method comparison</span>
        <span class="has-text-right">
          <i class="bi bi-lightbulb-fill"></i><i class="bi bi-stoplights medium"></i>
        </span>
    </div>
<div class="message-body">


Analyze the mean squared error (MSE), and inspect the adjusted pose sequences for each missing data approach and discuss the advantages and disadvantages of each method.


</div></article>



In [0]:
# Write your answers here

### Descriptive statistics for evaluating imputation method
As a final step, we need to evaluate how the missing value imputation procedure affects the statistical properties of the data. Specifically, we will focus on the mean and the covariance of both the original and the imputed poses.
<article class="message task"><a class="anchor" id="Mean"></a>
    <div class="message-header">
        <span>Task 6: Mean pose and pose sequence</span>
        <span class="has-text-right">
          <i class="bi bi-code"></i><i class="bi bi-lightbulb-fill"></i><i class="bi bi-stoplights medium"></i>
        </span>
    </div>
<div class="message-body">


The cell below contains code for plotting the original mean pose sequence. First run the cell, then modify the code to:
- Plot the mean sequence after filling in the missing values with zeros.
- Compare the two plots. Do you observe any differences? If yes, what could be the reason behind these differences and what insights can they provide regarding the characteristics of the mean?
- Plot the mean pose (not a sequence of poses, but a single pose) for the ground truth data and for the imputed datasets as well. Do you observe any differences between the mean pose (or the scale of the pose) in any of the plots?
- Based on the way in which the mean is calculated, is the mean sequence of poses or the mean pose more affected by the imputation process for missing data?



</div></article>



In [0]:
## Plotting the mean sequence
plot_single_sequence(GT_poses.mean(axis=(0)),color='red')

# Your solution goes here

In addition to analyzing the data's mean, we also need to investigate how the different imputation methods impact the relationship between variables. The tutorial demonstrated that correlation matrices are useful for quantifying how multiple variables co-vary. Throughout the next task we generate multiple correlation matrices to examine whether the relationship between features/joints is preserved by the way we handled missing data:
<article class="message task"><a class="anchor" id="Correlation"></a>
    <div class="message-header">
        <span>Task 7: Correlation between features</span>
        <span class="has-text-right">
          <i class="bi bi-code"></i><i class="bi bi-lightbulb-fill"></i><i class="bi bi-stoplights easy"></i>
        </span>
    </div>
<div class="message-body">


This task is divided into multiple steps, as we need to construct and compare multiple correlation matrices:
<article class="message is-info">
  <div class="message-header">Info</div>
  <div class="message-body">

  In the tutorial the correlation matrix was generated using the corr() function, which is exclusively applicable to Pandas DataFrames. In the current exercises we are working with a Numpy array. The correlation matrix for such an array can be obtained using the np.corrcoef() function.


  </div>
</article>

The cell below contains code for pre-processing the ground truth dataset and constructing the correlation matrix:
1. Run the code and modify it to obtain a correlation matrix for each of the imputed datasets.
2. Compare the correlation matrices:    - Do any of the imputation methods preserve the original relationships between joints?
    - Does it have any significance whether the original relationship is preserved?


3. Based on the MSE scores, the visual inspection, and the inspection of statistical properties, which imputation method would you chose for the current dataset and why? List at least 5 different reasons for your choice.



</div></article>



In [0]:
# pre-processing the data: flattening and transposing all of the ground truth poses
GT_poses_reshaped = GT_poses.reshape(-1,50)[:].T

GT_poses_reshaped_only_x = GT_poses.reshape(-1,50)[:,::2].T
GT_poses_reshaped_only_y = GT_poses.reshape(-1,50)[:,1::2].T

# Generating the correlation matrix
plt.title('Keypoint correlation heatmap plot',fontsize=22)
sns.heatmap(np.corrcoef(GT_poses_reshaped),cmap='coolwarm')
plt.show()
### Only x and only y correlation keypoint plots
plt.figure(figsize=(16,6))
plt.suptitle('Only 1 coordinate (x or y) keypoint correlation plot',fontsize=30)
plt.subplot(1,2,1)
sns.heatmap(np.corrcoef(GT_poses_reshaped_only_x),cmap='coolwarm')
plt.subplot(1,2,2)
sns.heatmap(np.corrcoef(GT_poses_reshaped_only_y),cmap='coolwarm')
plt.show();

In [0]:
### write reflection here ...